# Task Assignment Optimization Problem with Gurobi Framework

_by: TK-Bunga Matahari Team_

---

# 0. The Obligatory Part


In [1]:
# Import library
import numpy as np
import pandas as pd
import gurobipy as gp
import matplotlib.pyplot as plt
from gurobipy import GRB, quicksum
from competency_assessment import CompetencyAssessment

# 1. Define the Data Structure


In [2]:
# Run this if the data in Local/Repository
new_employee_path = "./mini_data/mini_data - employee.csv"
new_task_path = "./mini_data/mini_data - task.csv"

# new_employee_path = '/content/drive/MyDrive/Tadika Mesra Bunga Matahari/#1 Optimization Problem/project1_task-assignment/data/fixed/fixed_data_employee.csv'
# new_task_path = '/content/drive/MyDrive/Tadika Mesra Bunga Matahari/#1 Optimization Problem/project1_task-assignment/data/fixed/fixed_data_task.csv'

## 1.1. Pre-Processing: Employee Data


In [3]:
# Read data
employee_skills_df = pd.read_csv(new_employee_path, index_col="employee_id")
employee_skills_df.drop(columns=["No", "Role"], inplace=True, errors="ignore")

employees = employee_skills_df.index.tolist()
skills_name = employee_skills_df.columns[1:].tolist()

employee_skills_df

math  python  sql  cloud  database  optimization
employee_id                                                  
Talent 1        5       5    3      2         2             5
Talent 2        2       5    4      3         3             1
Talent 3        4       4    5      3         4             3
Talent 4        3       4    5      5         4             2
Talent 5        2       3    2      4         5             1

## 1.2. Pre-Processing: Task Data


In [4]:
task_df = pd.read_csv(new_task_path, index_col="task_id")

tasks = task_df.index.tolist()
company_names = list(set(task_df["project_id"]))
story_points = task_df["story_points"].to_dict()

task_df

project_id  story_points  math  python  sql  cloud  database   
task_id                                                                
T1              P2             1     0       5    3      4         4  \
T2              P1             2     3       5    4      5         3   
T3              P2             3     5       4    2      0         0   
T4              P3             5     2       4    0      3         0   
T5              P3             1     0       3    5      1         5   
T6              P1             1     0       1    4      5         5   
T7              P3             2     4       4    1      1         2   
T8              P1             3     1       0    5      5         3   
T9              P2             1     3       4    3      0         0   
T10             P1             3     5       5    0      0         0   

         optimization  
task_id                
T1                  0  
T2                  1  
T3                  5  
T4                  4  
T5                  0  
T6                  0  
T7                  3  
T8                  0  
T9                  2  
T10                 5

## 1.3. Group the task data by company/project


In [5]:
# convert to dictionary each company and its task
company_tasks = {}

for company in company_names:
    company_tasks[company] = task_df[task_df["project_id"] == company].index.tolist()

# sort the company tasks from C1 to C5
company_tasks = dict(sorted(company_tasks.items()))

company_tasks_df = pd.DataFrame.from_dict(company_tasks, orient="index")
company_tasks_df

0   1   2     3
P1  T2  T6  T8   T10
P2  T1  T3  T9  None
P3  T4  T5  T7  None

## 1.4. Pre-Processing: Competency Assesment

First, create RCD-ACD Dataframe that we get from Task Dataframe for RCD and from Employee Dataframe for ACD.

### 1.4.1 Required Competence Data


In [6]:
rcd_df = task_df.drop(columns=["project_id", "story_points"])
rcd_df = rcd_df.fillna(0)
rcd_df

math  python  sql  cloud  database  optimization
task_id                                                  
T1          0       5    3      4         4             0
T2          3       5    4      5         3             1
T3          5       4    2      0         0             5
T4          2       4    0      3         0             4
T5          0       3    5      1         5             0
T6          0       1    4      5         5             0
T7          4       4    1      1         2             3
T8          1       0    5      5         3             0
T9          3       4    3      0         0             2
T10         5       5    0      0         0             5

### 1.4.2 Acquired Competence Data


In [7]:
# create a copy of the original DataFrame
acd_df = employee_skills_df.copy()
acd_df = acd_df.fillna(0)
acd_df

math  python  sql  cloud  database  optimization
employee_id                                                  
Talent 1        5       5    3      2         2             5
Talent 2        2       5    4      3         3             1
Talent 3        4       4    5      3         4             3
Talent 4        3       4    5      5         4             2
Talent 5        2       3    2      4         5             1

### 1.4.3 Fit the Data


In [8]:
ca = CompetencyAssessment(rcd_df, acd_df)
qs, info = ca.fit()

### 1.4.4 Qualification Space


In [9]:
qs

{'Talent 1': {'T1': [0.0, -1.0, -0.16666666666666666, 'Under-Qualified'],
  'T2': [0.4761904761904761,
   -1.0476190476190477,
   -0.09523809523809527,
   'Under-Qualified'],
  'T3': [0.375, 0, 0.0625, 'Qualified'],
  'T4': [1.076923076923077,
   -0.23076923076923073,
   0.14102564102564105,
   'Qualified'],
  'T5': [0.5, -1.785714285714286, -0.21428571428571433, 'Under-Qualified'],
  'T6': [0.26666666666666666,
   -2.2666666666666666,
   -0.3333333333333333,
   'Under-Qualified'],
  'T7': [1.133333333333333, 0, 0.18888888888888886, 'Qualified'],
  'T8': [0.2857142857142857,
   -2.0000000000000004,
   -0.2857142857142858,
   'Under-Qualified'],
  'T9': [1.3333333333333333, 0, 0.2222222222222222, 'Qualified'],
  'T10': [0.0, 0, 0.0, 'Qualified']},
 'Talent 2': {'T1': [0.1875, -0.5, -0.052083333333333336, 'Under-Qualified'],
  'T2': [0.0, -0.6190476190476191, -0.10317460317460318, 'Under-Qualified'],
  'T3': [0.5, -2.1875, -0.28125, 'Under-Qualified'],
  'T4': [0.3076923076923077,
   -0.

### 1.4.5 Sorted MSG Score for All Tasks


In [10]:
score = ca.rank_MSG(qs)
score_for_obj2 = score
score

{'Talent 1': {'T9': 0.2222222222222222,
  'T7': 0.18888888888888886,
  'T4': 0.14102564102564105,
  'T3': 0.0625,
  'T10': 0.0,
  'T2': -0.09523809523809527,
  'T1': -0.16666666666666666,
  'T5': -0.21428571428571433,
  'T8': -0.2857142857142858,
  'T6': -0.3333333333333333},
 'Talent 2': {'T9': 0.027777777777777773,
  'T7': -0.033333333333333326,
  'T1': -0.052083333333333336,
  'T5': -0.08333333333333337,
  'T4': -0.10256410256410257,
  'T2': -0.10317460317460318,
  'T8': -0.1666666666666667,
  'T6': -0.17777777777777773,
  'T3': -0.28125,
  'T10': -0.38888888888888884},
 'Talent 3': {'T9': 0.1527777777777778,
  'T7': 0.11111111111111112,
  'T4': 0.0,
  'T5': -9.25185853854297e-18,
  'T2': -0.023809523809523836,
  'T1': -0.03125,
  'T8': -0.04761904761904764,
  'T6': -0.08888888888888886,
  'T3': -0.09375,
  'T10': -0.22222222222222218},
 'Talent 4': {'T9': 0.08333333333333333,
  'T8': 0.05952380952380953,
  'T7': 0.05555555555555556,
  'T1': 0.052083333333333336,
  'T2': 0.023809523

# 2. Construct the Model


In [11]:
WLSACCESSID = "f26730de-b14b-4197-9dbd-7d12372b5d9e"
WLSSECRET = "4ff3e9d2-037b-47c4-898d-6de34404e994"
LICENSEID = 2521640

In [12]:
# Create an environment with WLS license
params = {
    "WLSACCESSID": WLSACCESSID,
    "WLSSECRET": WLSSECRET,
    "LICENSEID": LICENSEID,
}
env = gp.Env(params=params)

# Create the model within the Gurobi environment
model = gp.Model(name="task_assignment", env=env)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2521640
Academic license 2521640 - for non-commercial use only - registered to na___@mail.ugm.ac.id


# 3. Build the Decision Variable

We have 3 sets:

$$
\text{sets} = \begin{cases}
I &: \text{set of tasks} \\
J &: \text{set of employees} \\
K &: \text{set of projects}
\end{cases}
$$

Next, we define parameters, scalars, and data structures. Let:

$$
\begin{align*}
i & = \text{task } i \\
j & = \text{employee } j \\
k & = \text{project } k \\
s_i & = \text{story points of task } i \\
e_{ij} & = \text{similarity skills of employee } j \text{ for task } i \\
\end{align*}
$$

**Decision Variables:**

$$
\begin{align*}
x_{ijk} & = \text{Binary variable indicating whether employee } j \text{ is assigned to task } k \text{ for day } i \\
y_{jk} & = \text{Binary variable indicating whether employee } j \text{ is assigned to any task from company } k \\
\end{align*}
$$


In [13]:
max_employee_workload = 20

In [14]:
# Create decision variables for x and y
x = {}
for k, task in company_tasks.items():
    for i in task:
        for j in employees:
            x[(i, j, k)] = model.addVar(vtype=GRB.BINARY, name=f"x_{i}_{j}_{k}")

# Decision variable y to represent cardinality of each employee and company
y = {}
for j in employees:
    for k in company_tasks.keys():
        y[(j, k)] = model.addVar(vtype=GRB.BINARY, name=f"y_{j}_{k}")

# Decision variable for max workload
max_workload = model.addVar(
    vtype=GRB.INTEGER, lb=0, ub=max_employee_workload, name="max_workload"
)

# Integrate new variables
model.update()

In [15]:
print(x)
print(y)

{('T2', 'Talent 1', 'P1'): <gurobi.Var x_T2_Talent 1_P1>, ('T2', 'Talent 2', 'P1'): <gurobi.Var x_T2_Talent 2_P1>, ('T2', 'Talent 3', 'P1'): <gurobi.Var x_T2_Talent 3_P1>, ('T2', 'Talent 4', 'P1'): <gurobi.Var x_T2_Talent 4_P1>, ('T2', 'Talent 5', 'P1'): <gurobi.Var x_T2_Talent 5_P1>, ('T6', 'Talent 1', 'P1'): <gurobi.Var x_T6_Talent 1_P1>, ('T6', 'Talent 2', 'P1'): <gurobi.Var x_T6_Talent 2_P1>, ('T6', 'Talent 3', 'P1'): <gurobi.Var x_T6_Talent 3_P1>, ('T6', 'Talent 4', 'P1'): <gurobi.Var x_T6_Talent 4_P1>, ('T6', 'Talent 5', 'P1'): <gurobi.Var x_T6_Talent 5_P1>, ('T8', 'Talent 1', 'P1'): <gurobi.Var x_T8_Talent 1_P1>, ('T8', 'Talent 2', 'P1'): <gurobi.Var x_T8_Talent 2_P1>, ('T8', 'Talent 3', 'P1'): <gurobi.Var x_T8_Talent 3_P1>, ('T8', 'Talent 4', 'P1'): <gurobi.Var x_T8_Talent 4_P1>, ('T8', 'Talent 5', 'P1'): <gurobi.Var x_T8_Talent 5_P1>, ('T10', 'Talent 1', 'P1'): <gurobi.Var x_T10_Talent 1_P1>, ('T10', 'Talent 2', 'P1'): <gurobi.Var x_T10_Talent 2_P1>, ('T10', 'Talent 3', 'P1'):

# 4. Subject to the Constraint

## 4.1. Constraint 1: Each Task is Assigned to One Employee

$$
\sum _{j\in J}\:x_{ijk}\:=\:1 \quad \forall i \in k, \: k \in K
$$


In [16]:
# constraint 1: each task assigned to one talent
for k, task in company_tasks.items():
    for i in task:
        model.addConstr(quicksum(x[(i, j, k)] for j in employees) == 1)

## 4.2. Constraint 2: Each employee works for one company at a time

Pre-Processing for Constraint 2:

$$
\sum _{i\in I_k}x_{ijk} > 0 \: \rightarrow \: y_{jk}=1 \quad \forall j\in J, \: k\in K\:
$$


In [17]:
# pre-processing constraint 2
for j in employees:
    for k, task in company_tasks.items():
        # Use quicksum to sum up x[i][j][k] for all i
        temp_sum = quicksum(x[i, j, k] for i in task)

        # Add a constraint to the model: y[j][k] is 1 if the sum of x[i][j][k] for all i is > 0, and 0 otherwise
        model.addGenConstrIndicator(y[j, k], True, temp_sum, GRB.GREATER_EQUAL, 1)
        model.addGenConstrIndicator(y[j, k], False, temp_sum, GRB.LESS_EQUAL, 0)

$$
\sum _{k\in K}y_{jk}\le 1 \quad \forall j\in J
$$


In [18]:
# create constraint 2: each employee can only work on one task
for j in employees:
    # The sum of y[j][k] for all companies (k) should be <= 1
    model.addConstr(quicksum(y[(j, k)] for k in company_tasks.keys()) <= 1)

## 4.3. Constraint 3: Employee workload doesn't exceed the capacity

$$
\sum _{i \in I} s_i \cdot x_{ijk} \le max\_workload \quad \forall j\in J, \: k\in K
$$


In [19]:
for j in employees:
    for k, tasks in company_tasks.items():
        model.addConstr(
            quicksum(story_points[i] * x[(i, j, k)] for i in tasks) <= max_employee_workload
        )

## 4.4 Constraint 4: Maximum workload is greater than or equal to the workload of each employee For Objective 3

$$
max\_workload \ge \sum_{i \in I} \sum_{k \in K} s_i\cdot x_{ijk}, \quad \forall j\in J\:\:
$$


In [20]:
# constraint 4: max_workload is greater than or equal to the workload of each employee
for j in employees:
    model.addConstr(
        max_workload
        >= quicksum(
            story_points[i] * x[i, j, k]
            for k, tasks in company_tasks.items()
            for i in tasks
        )
    )